In [1]:
import json
import sys
from pathlib import Path
import numpy as np
import h5py
import trimesh
import matplotlib.pyplot as plt
notebook_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/notebooks")

In [ ]:
pc_path = Path("/home/pauldelseith/dataset_storage/partnet/ins_seg_h5/Chair/train-00.h5")
sem_pc_path = Path("/home/pauldelseith/dataset_storage/partnet/sem_seg_h5/Chair-1/train-00.h5")
anno_path = Path("/home/pauldelseith/dataset_storage/partnet/ins_seg_h5/Chair/train-00.json")
with h5py.File(pc_path, 'r') as f:
    print(f.keys())
    points = f["pts"][:]
    color = f["rgb"][:]
    print(color.shape)
with h5py.File(sem_pc_path, 'r') as f:
    print(f.keys())
    points_sem = f["data"][:]
    color_sem = f["label_seg"][:]
    print(color.shape)
with open(anno_path, 'r') as f:
    anno = json.load(f)


In [ ]:
color_sem

In [ ]:
shape_id = "39524"
for idx, shape in enumerate(anno):
    if shape["anno_id"] == shape_id:
        print(idx)
        break
shape_color = color[idx]
shape_points = points[idx]

In [ ]:
shape_color

In [ ]:
trimesh.points.PointCloud(shape_points, colors=shape_color).export(
                notebook_dir / f"{shape_id}_pc.obj"
            )

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot: 'c' for color, 'marker' for point shape
scatter = ax.scatter(shape_points[:, 0], shape_points[:, 1], shape_points[:, 2], c=shape_color / 255, marker='o')

# Labeling the axes
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')

# Title of the plot
ax.set_title('3D Scatter Plot')

In [2]:
sys.path.append(
    "/home/pauldelseith/HyperDiffusion_semantic_decomposition"
)

In [3]:
from helpers import HYPER_DIFF_DIR
from siren.experiment_scripts.partnet_pc_generation import sample_occupancy_grid_from_mesh
from data.partnet_semantic_mesh_merging import normalize_mesh

In [ ]:
mesh = trimesh.load_mesh("/home/pauldelseith/HyperDiffusion_semantic_decomposition/notebooks/new-3.obj")
mesh = normalize_mesh(mesh)[0]

In [ ]:
points, occupancies = sample_occupancy_grid_from_mesh(mesh, 100000, 100000)

In [ ]:
colors = np.zeros((points.shape[0], 4), dtype=np.uint8)
colors[:, 3] = 1
colors[occupancies[:,0] == 1, 0] = 255
colors[occupancies[:,0] == 0, 1] = 255

In [ ]:
trimesh.points.PointCloud(points, colors=colors).export(
                notebook_dir / f"occupancy_grid_{shape_id}.obj"
            )

In [ ]:
mesh.export(notebook_dir / "new-3_normalized.obj")

In [ ]:
from hyperdiffusion import HyperDiffusion